In [ ]:
#Import necessary libraries
import numpy as np
import pandas as pd

#Load the datasets and Create an ExcelFile object
from google.colab import drive
drive.mount('/content/drive')
Data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/سالیانه زنجیره ای.xlsx')

In [ ]:
#Initial check
print(Data.head(5))
print('\n')
print(Data.dtypes)
print('\n')
print(Data.info())
print('\n')
print(Data.isnull().sum())
print('\n')
print(Data.duplicated().sum())

In [ ]:
#Preprocessing - chose the valuable columns

SelectedColumns = [ 'کد مشتری', 'کد فروشنده', 'کد کالا', 'نام مشتری', 'نام فروشگاه',
                    'فروشنده', 'نام کالا', 'گروه کالا', 'ماه', 'سال', 'مبلغ فروش خالص',
                    'قیمت واحد', 'تعداد فروش و جایزه-ظرف', 'تعداد فروش و جایزه-کارتن' ]

NewData = Data[SelectedColumns]
print(NewData.info())

In [ ]:
#Preprocessing - rename the headers

NewData.rename(columns={
               'کد مشتری': 'CustomerCode',
               'کد فروشنده': 'SellerCode',
               'کد کالا': 'ProductCode',
               'نام مشتری': 'CustomerName',
               'نام فروشگاه': 'StoreName',
               'فروشنده': 'SellerName',
               'نام کالا': 'ProductName',
               'گروه کالا': 'ProductGroup',
               'ماه': 'Month',
               'سال': 'Year',
               'مبلغ فروش خالص': 'NetSales',
               'قیمت واحد': 'UnitPrice',
               'تعداد فروش و جایزه-ظرف': 'Units',
               'تعداد فروش و جایزه-کارتن': 'Cartons'
               }, inplace=True)

print(NewData.info())

In [ ]:
#Preprocessing - function to normalize Persian text

import re

#Function to manage the text
def normalize_persian(text):
    if not isinstance(text, str):
        return None                     #Handle non-string values
    text = text.strip()                 #Remove leading/trailing spaces
    text = re.sub(r'\s+', ' ', text)    #Replace multiple spaces with one
    text = text.replace('ي', 'ی')       #Convert Arabic "ي" to Persian "ی"
    text = text.replace('ك', 'ک')       #Convert Arabic "ك" to Persian "ک"
    return text

PersianMonths = {'فروردین': '01','اردیبهشت': '02','خرداد': '03','تیر': '04',
                 'مرداد': '05','شهریور': '06','مهر': '07','آبان': '08',
                 'آذر': '09','دی': '10','بهمن': '11','اسفند': '12'}

#Normalize and debug 'Month' column
NewData['Month'] = NewData['Month'].apply(normalize_persian)
NewData['MonthNumeric'] = NewData['Month'].map(PersianMonths)
NewData['MonthNumeric'].fillna('Unknown', inplace=True)
NewData = NewData[NewData['MonthNumeric'] != 'Unknown']

In [ ]:
#Check
print("Unique values in 'Month':")
print(NewData['MonthNumeric'].unique())
print(NewData.info())

In [ ]:
#Preprocessing - reorder rows by year

NewData.sort_values(by='Year', ascending=True, inplace=True)
print(NewData['Year'].head(30))

In [ ]:
#Preprocessing - define numeric codes for unique values

NewData['StoreCode'] = NewData['StoreName'].astype('category').cat.codes
NewData['ProductGroupCode'] = NewData['ProductGroup'].astype('category').cat.codes
print(NewData.info())

In [ ]:
#Preprocessing - remove dots in specific columns

NewData['Cartons'] = NewData['Cartons'].astype(str).str.replace('.', '', regex=False)
NewData['Units'] = NewData['Units'].astype(str).str.replace('.', '', regex=False)

print(NewData[['Cartons', 'Units']].head(10))

In [ ]:
#Preprocessing - change types of numeric columns

NumericColumns = ['NetSales', 'UnitPrice', 'Units', 'Cartons']

for column in NumericColumns:
    NewData[column] = pd.to_numeric(NewData[column], errors='coerce')

print(NewData.info())

In [ ]:
#Preprocessing - keep identifiers as strings

IdentifierColumns = ['CustomerCode', 'SellerCode', 'ProductCode', 'StoreCode', 'ProductGroupCode']

for column in IdentifierColumns:
    NewData[column] = NewData[column].astype(str)

print(NewData.info())

In [ ]:
#Preprocessing - convert year and month columns to integers

NewData['Year'] = NewData['Year'].astype(int)
NewData['MonthNumeric'] = NewData['MonthNumeric'].astype(int)

print(NewData.info())

In [ ]:
#Check

print(NewData.isnull().sum())
print('\n')
print(NewData.duplicated().sum())

DuplicatesAllColumns = NewData[NewData.duplicated()]

KeyColumns = ['CustomerCode', 'SellerCode', 'ProductCode', 'Month', 'Year']
DuplicatesKeyColumns = NewData[NewData.duplicated(subset=KeyColumns)]

print("Duplicates based on all columns:")
print(DuplicatesAllColumns)

print("\nDuplicates based on selected key columns:")
print(DuplicatesKeyColumns)

In [ ]:
#Save the data to a dataframe and new csv file

FilePath = '/content/drive/MyDrive/Colab Notebooks/ChainCleaned.csv'
NewData.to_csv(FilePath, index=False)